# Environment Setup

In [ ]:
# Remova o diretório antes de clonar o repositório
!rm -r seg_depth

In [ ]:
# Clone o repositório
!git clone https://github.com/duraes-antonio/seg_depth

In [ ]:
# Mude para o diretório do repositório
%cd seg_depth

# Depth Estimation

## Prepare Dataset

In [ ]:
# Download dataset de treino + validação
!gdown 11E9hVtTqwRwPkKyEf_yKU3q_Gb0ngl2a

In [ ]:
!unzip -qq nyu_data.zip
!rm nyu_data.zip

In [ ]:
# Download dataset de teste
!gdown 1mNj2aC1ongKLkjaVRYArvbr26xwMEmJG

## Prepare networks weights

In [ ]:
folder_weight_name = 'pretrained.zip'

In [ ]:
# Download dos pesos do ViT-R50 para TransUnet
!gdown 1Krx3KE6JWAIScIZSsxevkySn_xOqLU5i

In [ ]:
!unzip -qq $folder_weight_name
!rm $folder_weight_name

## Train

In [ ]:
from options.dataset_resolution import Resolutions
from options.model import Models

batch_size = 8
workers = 2
epochs = 70
resolution = Resolutions.Half2
# clp = 'segmentation/unet_mt-b3/base_model-unet_mt-b3_factor-1_ds-usage-100_batch-2_opt-0_scheduler-0_\{epoch\}_model.pth'
clp = ''
model = Models.UNetMixedTransformerB2
run_dir = 'base_ds-100'

In [ ]:
!python lightning_main_depth.py -b $batch_size -w $workers -e $epochs -s $resolution -clp $clp -mr train -rd $run_dir -du 100 -m $model

# Upload files to Google Drive

In [ ]:
from infra.consts.environment import Environment

env = Environment.KAGGLE

In [ ]:
from google.auth.credentials import Credentials
from google.oauth2 import service_account
from googleapiclient.discovery import build

class GoogleDriveService:
    def __init__(self, credential_filename: str):
        credentials = self.__authenticate__(credential_filename)
        self.service = self.__build_service__(credentials)

    @staticmethod
    def __authenticate__(filename: str) -> Credentials:
        credentials = service_account.Credentials.from_service_account_info(
            filename, scopes=["https://www.googleapis.com/auth/drive"]
        )
        return credentials

    @staticmethod
    def __build_service__(credentials: Credentials):
        return build("drive", "v3", credentials=credentials)

    def upload_file(
            self,
            file_path: str,
            parent_folder_id: str,
            new_file_name: str = None
    ) -> str:
        file_metadata = {
            'name': new_file_name or file_path.split('/')[-1],
            'parents': [parent_folder_id]
        }
        file = self.service.files().create(
            body=file_metadata,
            media_body=file_path,
            fields='id'
        ).execute()
        return file.get('id')

In [ ]:
import ast, os
import dotenv

env_vars = {}

if env is Environment.KAGGLE:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    env_vars = {
        'google_credentials': ast.literal_eval(user_secrets.get_secret("GOOGLE_CREDENTIALS_JSON")),
    }

else:
    colab_env_path = '../drive/MyDrive/unet_depth/auth/.env'
    local_env_path = './.env'
    dotenv.load_dotenv(colab_env_path if env is Environment.COLAB else local_env_path)
    env_vars = {
        'google_credentials': ast.literal_eval(os.environ["GOOGLE_CREDENTIALS_JSON"]),
    }

In [ ]:
google_drive_service = GoogleDriveService(env_vars['google_credentials'])
google_drive_service.upload_file()